<a href="https://colab.research.google.com/github/geleshChrsitUniversity/nikitadurasov-masksembles/blob/main/notebooks/MNIST_Masksembles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade git+http://github.com/nikitadurasov/masksembles
!wget https://github.com/nikitadurasov/masksembles/raw/main/images/complex_sample_mnist.npy

  Cloning http://github.com/nikitadurasov/masksembles to /tmp/pip-req-build-7i7r68nz
  Running command git clone --filter=blob:none --quiet http://github.com/nikitadurasov/masksembles /tmp/pip-req-build-7i7r68nz
  Resolved http://github.com/nikitadurasov/masksembles to commit 4866e1b9f33c97c4004bf95daadfabb24bc00c90
  Preparing metadata (setup.py) ... done
  Created wheel for masksembles: filename=masksembles-1.1-py3-none-any.whl size=8307 sha256=9b7ff7dde2edb320162d766447281332f1032be0dd9308a203ae3100cc7d45ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-bkwp96vr/wheels/78/f9/4e/b68a4c2dcd69743e09d55e28a048c6cfc631718fea05dfd392
Successfully built masksembles
--2025-08-30 13:58:20--  https://github.com/nikitadurasov/masksembles/raw/main/images/complex_sample_mnist.npy
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ni

In [2]:
from google.colab import drive
import os

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Set folder path (change this to your desired folder)
folder_path = '/content/drive/MyDrive/MyProjectFolder'

# Step 3: Make sure the folder exists
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Step 4: Change current directory
os.chdir(folder_path)

# Step 5: Verify
print("Current directory:", os.getcwd())


Mounted at /content/drive
Current directory: /content/drive/MyDrive/MyProjectFolder


In [4]:
import tensorflow as tf
#from . import common
import masksembles.common as common  # Ensure absolute import


class Masksembles2D(tf.keras.layers.Layer):
    """
    :class:Masksembles2D is high-level class that implements Masksembles approach
    for 2-dimensional inputs (similar to :class:tensorflow.keras.layers.SpatialDropout1D).

    :param n: int, number of masks
    :param scale: float, scale parameter similar to *S* in [1]. Larger values decrease \
        subnetworks correlations but at the same time decrease capacity of every individual model.

    Shape:
        * Input: (N, H, W, C)
        * Output: (N, H, W, C) (same shape as input)

    Examples:

    >>> m = Masksembles2D(4, 2.0)
    >>> inputs = tf.ones([4, 28, 28, 16])
    >>> output = m(inputs)

    References:

    [1] Masksembles for Uncertainty Estimation,
    Nikita Durasov, Timur Bagautdinov, Pierre Baque, Pascal Fua

    """

    def __init__(self, n: int, scale: float):
        super(Masksembles2D, self).__init__()

        self.n = n
        self.scale = scale

    def build(self, input_shape):
        channels = input_shape[-1]
        masks = common.generation_wrapper(channels, self.n, self.scale)
        self.masks = self.add_weight(name="masks",
                                     shape=masks.shape,
                                     trainable=False,
                                     dtype="float32")
        self.masks.assign(masks)

    def call(self, inputs, training=False):
        # inputs : [N, H, W, C]
        # masks : [M, C]
        x = tf.stack(tf.split(inputs, self.n))
        # x : [M, N // M, H, W, C]
        # masks : [M, 1, 1, 1, C]
        x = x * self.masks[:, tf.newaxis, tf.newaxis, tf.newaxis]
        x = tf.concat(tf.split(x, self.n), axis=1)
        return tf.squeeze(x, axis=0)


class Masksembles1D(tf.keras.layers.Layer):
    """
    :class:Masksembles1D is high-level class that implements Masksembles approach
    for 1-dimensional inputs (similar to :class:tensorflow.keras.layers.Dropout).

    :param n: int, number of masks
    :param scale: float, scale parameter similar to *S* in [1]. Larger values decrease \
        subnetworks correlations but at the same time decrease capacity of every individual model.

    Shape:
        * Input: (N, C)
        * Output: (N, C) (same shape as input)

    Examples:

    >>> m = Masksembles1D(4, 2.0)
    >>> inputs = tf.ones([4, 16])
    >>> output = m(inputs)


    References:

    [1] Masksembles for Uncertainty Estimation,
    Nikita Durasov, Timur Bagautdinov, Pierre Baque, Pascal Fua

    """

    def __init__(self, n: int, scale: float):
        super(Masksembles1D, self).__init__()

        self.n = n
        self.scale = scale

    def build(self, input_shape):
        channels = input_shape[-1]
        masks = common.generation_wrapper(channels, self.n, self.scale)
        self.masks = self.add_weight(name="masks",
                                     shape=masks.shape,
                                     trainable=False,
                                     dtype="float32")
        self.masks.assign(masks)

    def call(self, inputs, training=False):
        x = tf.stack(tf.split(inputs, self.n))
        x = x * self.masks[:, tf.newaxis]
        x = tf.concat(tf.split(x, self.n), axis=1)
        return tf.squeeze(x, axis=0)

#LOOPED EXECUTION

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np

# =====================
# Utility: Models
# =====================
def make_plain_model(input_shape, num_classes):
    return keras.Sequential([
        keras.Input(shape=input_shape),
        layers.Conv2D(32, (3,3), activation="elu"),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation="elu"),
        layers.MaxPooling2D((2,2)),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax")
    ])

def make_dropout_model(input_shape, num_classes, drop_rate=0.35):
    return keras.Sequential([
        keras.Input(shape=input_shape),
        layers.Conv2D(32, (3,3), activation="elu"),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation="elu"),
        layers.MaxPooling2D((2,2)),
        layers.Flatten(),
        layers.Dropout(drop_rate),
        layers.Dense(num_classes, activation="softmax")
    ])

# requires masksembles package installed
from masksembles.keras import Masksembles2D, Masksembles1D
def make_masksembles_model(input_shape, num_classes, m=4, scale=2.0):
    return keras.Sequential([
        keras.Input(shape=input_shape),
        layers.Conv2D(32, (3,3), activation="elu"),
        Masksembles2D(m, scale),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation="elu"),
        Masksembles2D(m, scale),
        layers.MaxPooling2D((2,2)),
        layers.Flatten(),
        Masksembles1D(m, scale),
        layers.Dense(num_classes, activation="softmax")
    ])

# =====================
# Utility: Dataset loaders
# =====================
def load_mnist():
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
    x_train = x_train.astype("float32")/255.0
    x_test = x_test.astype("float32")/255.0
    x_train = np.expand_dims(x_train, -1)   # (28,28,1)
    x_test = np.expand_dims(x_test, -1)
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)
    return (x_train, y_train), (x_test, y_test), (28,28,1), 10

def load_cifar10():
    (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
    x_train = x_train.astype("float32")/255.0
    x_test = x_test.astype("float32")/255.0
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)
    return (x_train, y_train), (x_test, y_test), (32,32,3), 10

# =====================
# Experiment Runner
# =====================
def run_experiments():
    results = []

    datasets = {
        "MNIST": load_mnist,
        "CIFAR-10": load_cifar10
    }

    # CIFAR train ratios
    cifar_ratios = [0.2, 0.5, 0.7, 1.0]
    # MNIST train ratios (e.g. 10% to 100%)
    mnist_ratios = cifar_ratios #[0.1, 0.25, 0.5, 0.75, 1.0]

    model_builders = {
        "Plain": make_plain_model,
        "Dropout": make_dropout_model,
        "Masksembles": make_masksembles_model
    }

    epochs = 20
    batch_size = 128

    for dataset_name, loader in datasets.items():
        (x_train, y_train), (x_test, y_test), input_shape, num_classes = loader()

        if dataset_name == "CIFAR-10":
            ratios = cifar_ratios
        else:
            ratios = mnist_ratios

        for ratio in ratios:
            # subset training set
            n = int(len(x_train) * ratio)
            x_sub, y_sub = x_train[:n], y_train[:n]

            for model_name, builder in model_builders.items():
                print(f"Training {model_name} on {dataset_name} with {ratio*100:.0f}% data")

                model = builder(input_shape, num_classes)
                model.compile(optimizer="adam",
                              loss="categorical_crossentropy",
                              metrics=["accuracy"])

                hist = model.fit(
                    x_sub, y_sub,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.1,
                    verbose=0
                )

                # collect history
                for epoch in range(epochs):
                    results.append({
                        "Dataset": dataset_name,
                        "TrainRatio": ratio,
                        "Model": model_name,
                        "Epoch": epoch+1,
                        "TrainAcc": hist.history["accuracy"][epoch],
                        "TrainLoss": hist.history["loss"][epoch],
                        "ValAcc": hist.history["val_accuracy"][epoch],
                        "ValLoss": hist.history["val_loss"][epoch],
                    })
                pd.DataFrame(results).to_csv(f"{dataset_name}_{ratio}_{model_name}.csv", index=False)


    df = pd.DataFrame(results)
    return df

# =====================
# Run and Save
# =====================
df_results = run_experiments()
df_results.to_csv("all_experiment_results.csv", index=False)
df_results.head()


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training Plain on MNIST with 20% data
Training Dropout on MNIST with 20% data
Training Masksembles on MNIST with 20% data


TypeError: Layer.add_weight() got multiple values for argument 'shape'

# MNIST

## Keras

In [3]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

import matplotlib.pyplot as plt

In [4]:
from masksembles.keras import Masksembles2D, Masksembles1D

In [5]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In order to transform regular model into Masksembles model one should add Masksembles2D or Masksembles1D layers in it. General recommendation is to insert these layers right before or after convolutional layers.

In example below we'll use both Masksembles2D and Masksembles1D layers applied after convolutions.

In [7]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="elu"),
        Masksembles2D(4, 2.0), # adding Masksembles2D
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Conv2D(64, kernel_size=(3, 3), activation="elu"),
        Masksembles2D(4, 2.0), # adding Masksembles2D
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Flatten(),
        Masksembles1D(4, 2.), # adding Masksembles1D
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

TypeError: Layer.add_weight() got multiple values for argument 'shape'

Training of Masksembles is not different from training of regular model. So we just use standard fit Keras API.

In [8]:
import tensorflow as tf
#from . import common
import masksembles.common as common  # Ensure absolute import


class Masksembles2D(tf.keras.layers.Layer):
    """
    :class:Masksembles2D is high-level class that implements Masksembles approach
    for 2-dimensional inputs (similar to :class:tensorflow.keras.layers.SpatialDropout1D).

    :param n: int, number of masks
    :param scale: float, scale parameter similar to *S* in [1]. Larger values decrease \
        subnetworks correlations but at the same time decrease capacity of every individual model.

    Shape:
        * Input: (N, H, W, C)
        * Output: (N, H, W, C) (same shape as input)

    Examples:

    >>> m = Masksembles2D(4, 2.0)
    >>> inputs = tf.ones([4, 28, 28, 16])
    >>> output = m(inputs)

    References:

    [1] Masksembles for Uncertainty Estimation,
    Nikita Durasov, Timur Bagautdinov, Pierre Baque, Pascal Fua

    """

    def __init__(self, n: int, scale: float):
        super(Masksembles2D, self).__init__()

        self.n = n
        self.scale = scale

    def build(self, input_shape):
        channels = input_shape[-1]
        masks = common.generation_wrapper(channels, self.n, self.scale)
        self.masks = self.add_weight(name="masks",
                                     shape=masks.shape,
                                     trainable=False,
                                     dtype="float32")
        self.masks.assign(masks)

    def call(self, inputs, training=False):
        # inputs : [N, H, W, C]
        # masks : [M, C]
        x = tf.stack(tf.split(inputs, self.n))
        # x : [M, N // M, H, W, C]
        # masks : [M, 1, 1, 1, C]
        x = x * self.masks[:, tf.newaxis, tf.newaxis, tf.newaxis]
        x = tf.concat(tf.split(x, self.n), axis=1)
        return tf.squeeze(x, axis=0)


class Masksembles1D(tf.keras.layers.Layer):
    """
    :class:Masksembles1D is high-level class that implements Masksembles approach
    for 1-dimensional inputs (similar to :class:tensorflow.keras.layers.Dropout).

    :param n: int, number of masks
    :param scale: float, scale parameter similar to *S* in [1]. Larger values decrease \
        subnetworks correlations but at the same time decrease capacity of every individual model.

    Shape:
        * Input: (N, C)
        * Output: (N, C) (same shape as input)

    Examples:

    >>> m = Masksembles1D(4, 2.0)
    >>> inputs = tf.ones([4, 16])
    >>> output = m(inputs)


    References:

    [1] Masksembles for Uncertainty Estimation,
    Nikita Durasov, Timur Bagautdinov, Pierre Baque, Pascal Fua

    """

    def __init__(self, n: int, scale: float):
        super(Masksembles1D, self).__init__()

        self.n = n
        self.scale = scale

    def build(self, input_shape):
        channels = input_shape[-1]
        masks = common.generation_wrapper(channels, self.n, self.scale)
        self.masks = self.add_weight(name="masks",
                                     shape=masks.shape,
                                     trainable=False,
                                     dtype="float32")
        self.masks.assign(masks)

    def call(self, inputs, training=False):
        x = tf.stack(tf.split(inputs, self.n))
        x = x * self.masks[:, tf.newaxis]
        x = tf.concat(tf.split(x, self.n), axis=1)
        return tf.squeeze(x, axis=0)

In [9]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="elu"),
        Masksembles2D(4, 2.0), # adding Masksembles2D
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Conv2D(64, kernel_size=(3, 3), activation="elu"),
        Masksembles2D(4, 2.0), # adding Masksembles2D
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Flatten(),
        Masksembles1D(4, 2.), # adding Masksembles1D
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ masksembles2d_4 (Masksembles2D) │ (None, 26, 26, 32)     │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ masksembles2d_5 (Masksembles2D) │ (None, 11, 11, 64)     │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ masksembles1d_2 (Masksembles1D) │ (None, 1600)           │         6,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │        16,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 41,610 (162.54 KB)

 Trainable params: 34,826 (136.04 KB)

 Non-trainable params: 6,784 (26.50 KB)

In [10]:
batch_size = 128
epochs = 20

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
trainHist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.5)

Epoch 1/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 13s 27ms/step - accuracy: 0.6234 - loss: 1.2848 - val_accuracy: 0.9157 - val_loss: 0.2838
Epoch 2/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9265 - loss: 0.2557 - val_accuracy: 0.9406 - val_loss: 0.1999
Epoch 3/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9471 - loss: 0.1758 - val_accuracy: 0.9533 - val_loss: 0.1586
Epoch 4/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9574 - loss: 0.1411 - val_accuracy: 0.9607 - val_loss: 0.1328
Epoch 5/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9665 - loss: 0.1169 - val_accuracy: 0.9645 - val_loss: 0.1193
Epoch 6/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9686 - loss: 0.0991 - val_accuracy: 0.9682 - val_loss: 0.1078
Epoch 7/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9743 - loss: 0.0882 - val_accuracy: 0.9672 - val_loss: 0.1067
Epoch 8/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9746 - loss: 0.0820 - val_accuracy: 

In [13]:
import pandas as pd
histDf = pd.DataFrame(trainHist.history)
histDf.sort_values("val_accuracy", ascending=False).head()

,accuracy,loss,val_accuracy,val_loss
19,0.985900,0.043972,0.978567,0.073463
17,0.985400,0.046898,0.976800,0.080573
16,0.985133,0.049130,0.976567,0.080647
18,0.985833,0.045645,0.976367,0.076756
15,0.984267,0.050202,0.976200,0.080168


In [14]:
histDf.sort_values("val_loss").head()

,accuracy,loss,val_accuracy,val_loss
19,0.985900,0.043972,0.978567,0.073463
18,0.985833,0.045645,0.976367,0.076756
14,0.983500,0.054319,0.975600,0.079918
15,0.984267,0.050202,0.976200,0.080168
17,0.985400,0.046898,0.976800,0.080573


In [17]:
modelPlain = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="elu"),
        #Masksembles2D(4, 2.0), # adding Masksembles2D
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Conv2D(64, kernel_size=(3, 3), activation="elu"),
        #Masksembles2D(4, 2.0), # adding Masksembles2D
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Flatten(),
        #layers.Dropout(0.35),
        #Masksembles1D(4, 2.), # adding Masksembles1D
        layers.Dense(num_classes, activation="softmax"),
    ]
)

modelPlain.summary()
modelPlain.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
trainHistPlain = modelPlain.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
histDfPlain = pd.DataFrame(trainHistPlain.history)
histDfPlain.sort_values("val_accuracy", ascending=False).head()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │        16,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,826 (136.04 KB)

 Trainable params: 34,826 (136.04 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.8237 - loss: 0.6356 - val_accuracy: 0.9763 - val_loss: 0.0808
Epoch 2/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9750 - loss: 0.0824 - val_accuracy: 0.9860 - val_loss: 0.0543
Epoch 3/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9829 - loss: 0.0557 - val_accuracy: 0.9847 - val_loss: 0.0533
Epoch 4/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9849 - loss: 0.0457 - val_accuracy: 0.9853 - val_loss: 0.0493
Epoch 5/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9885 - loss: 0.0368 - val_accuracy: 0.9872 - val_loss: 0.0508
Epoch 6/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9897 - loss: 0.0324 - val_accuracy: 0.9847 - val_loss: 0.0489
Epoch 7/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9909 - loss: 0.0291 - val_accuracy: 0.9885 - val_loss: 0.0429
Epoch 8/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9920 - loss: 0.0240 - val_accuracy: 0

,accuracy,loss,val_accuracy,val_loss
7,0.991148,0.027220,0.989500,0.041627
10,0.994704,0.018031,0.989167,0.047861
18,0.998444,0.005536,0.989167,0.054162
17,0.997759,0.006304,0.989000,0.053941
14,0.996537,0.010825,0.988833,0.054005


In [18]:
histDfPlain.sort_values("val_loss", ascending=True).head()

,accuracy,loss,val_accuracy,val_loss
7,0.991148,0.027220,0.989500,0.041627
6,0.990630,0.030334,0.988500,0.042883
11,0.995167,0.015087,0.987833,0.047591
10,0.994704,0.018031,0.989167,0.047861
9,0.993444,0.020668,0.988333,0.048003


In [15]:
modelDropOut = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="elu"),
        #Masksembles2D(4, 2.0), # adding Masksembles2D
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Conv2D(64, kernel_size=(3, 3), activation="elu"),
        #Masksembles2D(4, 2.0), # adding Masksembles2D
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Flatten(),
        layers.Dropout(0.35),
        #Masksembles1D(4, 2.), # adding Masksembles1D
        layers.Dense(num_classes, activation="softmax"),
    ]
)

modelDropOut.summary()
modelDropOut.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
trainHistDropOut = modelDropOut.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
histDfDropOut = pd.DataFrame(trainHistDropOut.history)
histDfDropOut.sort_values("val_accuracy", ascending=False).head()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │        16,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,826 (136.04 KB)

 Trainable params: 34,826 (136.04 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.8033 - loss: 0.6742 - val_accuracy: 0.9802 - val_loss: 0.0776
Epoch 2/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9682 - loss: 0.1031 - val_accuracy: 0.9853 - val_loss: 0.0534
Epoch 3/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9772 - loss: 0.0754 - val_accuracy: 0.9865 - val_loss: 0.0491
Epoch 4/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9797 - loss: 0.0639 - val_accuracy: 0.9860 - val_loss: 0.0509
Epoch 5/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9834 - loss: 0.0545 - val_accuracy: 0.9870 - val_loss: 0.0475
Epoch 6/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9837 - loss: 0.0513 - val_accuracy: 0.9880 - val_loss: 0.0454
Epoch 7/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9851 - loss: 0.0466 - val_accuracy: 0.9862 - val_loss: 0.0487
Epoch 8/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9859 - loss: 0.0453 - val_accuracy: 0

,accuracy,loss,val_accuracy,val_loss
18,0.990852,0.026865,0.990833,0.041671
16,0.990593,0.029586,0.990500,0.039636
19,0.990944,0.026293,0.990167,0.040026
12,0.989222,0.034531,0.989500,0.040406
15,0.990000,0.030081,0.989500,0.040423


In [16]:
histDfDropOut.sort_values("val_loss").head()

,accuracy,loss,val_accuracy,val_loss
11,0.988685,0.034901,0.989167,0.038226
14,0.989648,0.029668,0.989500,0.039352
16,0.990593,0.029586,0.990500,0.039636
10,0.987833,0.037416,0.989167,0.039927
19,0.990944,0.026293,0.990167,0.040026


In [ ]:
print('MaskEnsemble -\n',histDf.sort_values("val_loss").head(5))
print()
print('DropOut      -\n',histDfDropOut.sort_values("val_loss").head(5))


MaskEnsemble -
     accuracy      loss  val_accuracy  val_loss
12  0.985204  0.047907      0.986833  0.047192
16  0.987333  0.040537      0.986000  0.048979
19  0.989018  0.034942      0.985500  0.049971
13  0.986796  0.043706      0.985167  0.050993
14  0.986222  0.043265      0.985167  0.051626

DropOut      -
     accuracy      loss  val_accuracy  val_loss
17  0.990981  0.026739      0.991333  0.034643
16  0.990333  0.029325      0.991167  0.035324
15  0.989259  0.031351      0.991333  0.037294
12  0.988815  0.033145      0.989333  0.037572
19  0.991037  0.025974      0.990500  0.038096


In [ ]:
modelDropOut = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="elu"),
        #Masksembles2D(4, 2.0), # adding Masksembles2D
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Conv2D(64, kernel_size=(3, 3), activation="elu"),
        #Masksembles2D(4, 2.0), # adding Masksembles2D
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Flatten(),
        layers.Dropout(0.5),
        #Masksembles1D(4, 2.), # adding Masksembles1D
        layers.Dense(num_classes, activation="softmax"),
    ]
)

modelDropOut.summary()
modelDropOut.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
trainHistDropOut = modelDropOut.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
histDfDropOut = pd.DataFrame(trainHistDropOut.history)
histDfDropOut.sort_values("val_accuracy", ascending=False).head()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_20 (Conv2D)                   │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_20 (MaxPooling2D)      │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_21 (Conv2D)                   │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_21 (MaxPooling2D)      │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_10 (Flatten)                 │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 10)                  │          16,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 34,826 (136.04 KB)

 Trainable params: 34,826 (136.04 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.7850 - loss: 0.7167 - val_accuracy: 0.9787 - val_loss: 0.0824
Epoch 2/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9634 - loss: 0.1176 - val_accuracy: 0.9837 - val_loss: 0.0618
Epoch 3/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9737 - loss: 0.0848 - val_accuracy: 0.9857 - val_loss: 0.0512
Epoch 4/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9777 - loss: 0.0696 - val_accuracy: 0.9860 - val_loss: 0.0490
Epoch 5/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9802 - loss: 0.0657 - val_accuracy: 0.9862 - val_loss: 0.0497
Epoch 6/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9817 - loss: 0.0592 - val_accuracy: 0.9865 - val_loss: 0.0461
Epoch 7/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9813 - loss: 0.0588 - val_accuracy: 0.9875 - val_loss: 0.0442
Epoch 8/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9834 - loss: 0.0530 - val_accuracy: 0.

,accuracy,loss,val_accuracy,val_loss
18,0.988278,0.037323,0.991167,0.039466
13,0.985685,0.043890,0.990500,0.037998
15,0.986185,0.043503,0.990167,0.037168
12,0.985444,0.045017,0.990167,0.041489
10,0.985463,0.047088,0.990000,0.039702


In [ ]:
print('MaskEnsemble -\n',histDf.sort_values("val_loss").head(5))
print()
print('DropOut 0.5  -\n',histDfDropOut.sort_values("val_loss").head(5))

MaskEnsemble -
     accuracy      loss  val_accuracy  val_loss
12  0.985204  0.047907      0.986833  0.047192
16  0.987333  0.040537      0.986000  0.048979
19  0.989018  0.034942      0.985500  0.049971
13  0.986796  0.043706      0.985167  0.050993
14  0.986222  0.043265      0.985167  0.051626

DropOut 0.5  -
     accuracy      loss  val_accuracy  val_loss
15  0.986185  0.043503      0.990167  0.037168
13  0.985685  0.043890      0.990500  0.037998
16  0.986574  0.041164      0.989667  0.038462
18  0.988278  0.037323      0.991167  0.039466
10  0.985463  0.047088      0.990000  0.039702


In [ ]:
# CIF

# CIFAR 10

In [19]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Normalize pixel values to range [0,1]
x_train, x_test = x_train / 255.0, x_test / 255.0

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

input_shape=(32, 32, 3)

In [20]:
modelMasksemblesCifar = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="elu"),
        Masksembles2D(4, 2.0), # adding Masksembles2D
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Conv2D(64, kernel_size=(3, 3), activation="elu"),
        Masksembles2D(4, 2.0), # adding Masksembles2D
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Flatten(),
        #layers.Dropout(0.35),
        Masksembles1D(4, 2.), # adding Masksembles1D
        layers.Dense(num_classes, activation="softmax"),
    ]
)

modelMasksemblesCifar.summary()
modelMasksemblesCifar.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
trainHistmodelMasksemblesCifar35Cifar = modelMasksemblesCifar.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
histDfmodelMasksemblesCifar35Cifar = pd.DataFrame(trainHistmodelMasksemblesCifar35Cifar.history)
histDfmodelMasksemblesCifar35Cifar.sort_values("val_accuracy", ascending=False).head()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ masksembles2d_6 (Masksembles2D) │ (None, 30, 30, 32)     │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ masksembles2d_7 (Masksembles2D) │ (None, 13, 13, 64)     │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ masksembles1d_3 (Masksembles1D) │ (None, 2304)           │         9,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │        23,050 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 52,042 (203.29 KB)

 Trainable params: 42,442 (165.79 KB)

 Non-trainable params: 9,600 (37.50 KB)

Epoch 1/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.2612 - loss: 2.0241 - val_accuracy: 0.4292 - val_loss: 1.6424
Epoch 2/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4273 - loss: 1.6148 - val_accuracy: 0.4724 - val_loss: 1.5081
Epoch 3/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4782 - loss: 1.4857 - val_accuracy: 0.5116 - val_loss: 1.3919
Epoch 4/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5089 - loss: 1.3940 - val_accuracy: 0.5224 - val_loss: 1.3562
Epoch 5/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5376 - loss: 1.3309 - val_accuracy: 0.5580 - val_loss: 1.2881
Epoch 6/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5522 - loss: 1.2847 - val_accuracy: 0.5546 - val_loss: 1.2685
Epoch 7/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5667 - loss: 1.2498 - val_accuracy: 0.5776 - val_loss: 1.2237
Epoch 8/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5809 - loss: 1.2081 - val_accuracy: 

,accuracy,loss,val_accuracy,val_loss
17,0.633467,1.064307,0.6286,1.087579
19,0.636733,1.045267,0.6204,1.078784
14,0.622978,1.092679,0.6180,1.102152
15,0.626178,1.080322,0.6176,1.098366
18,0.636467,1.053379,0.6174,1.093312


In [21]:
histDfmodelMasksemblesCifar35Cifar.sort_values("val_loss").head()

,accuracy,loss,val_accuracy,val_loss
19,0.636733,1.045267,0.6204,1.078784
17,0.633467,1.064307,0.6286,1.087579
18,0.636467,1.053379,0.6174,1.093312
15,0.626178,1.080322,0.6176,1.098366
14,0.622978,1.092679,0.6180,1.102152


In [22]:
modelDropOut35Cifar = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="elu"),
        #Masksembles2D(4, 2.0), # adding Masksembles2D
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Conv2D(64, kernel_size=(3, 3), activation="elu"),
        #Masksembles2D(4, 2.0), # adding Masksembles2D
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Flatten(),
        layers.Dropout(0.35),
        #Masksembles1D(4, 2.), # adding Masksembles1D
        layers.Dense(num_classes, activation="softmax"),
    ]
)

modelDropOut35Cifar.summary()
modelDropOut35Cifar.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
trainHistDropOut35Cifar = modelDropOut35Cifar.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
histDfDropOut35Cifar = pd.DataFrame(trainHistDropOut35Cifar.history)
histDfDropOut35Cifar.sort_values("val_accuracy", ascending=False).head()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 10)             │        23,050 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,442 (165.79 KB)

 Trainable params: 42,442 (165.79 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - accuracy: 0.3357 - loss: 1.8544 - val_accuracy: 0.5188 - val_loss: 1.3884
Epoch 2/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5131 - loss: 1.3815 - val_accuracy: 0.5536 - val_loss: 1.2897
Epoch 3/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5604 - loss: 1.2724 - val_accuracy: 0.5914 - val_loss: 1.1923
Epoch 4/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5837 - loss: 1.2006 - val_accuracy: 0.6228 - val_loss: 1.1117
Epoch 5/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6039 - loss: 1.1493 - val_accuracy: 0.6282 - val_loss: 1.0811
Epoch 6/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6195 - loss: 1.1038 - val_accuracy: 0.6298 - val_loss: 1.0769
Epoch 7/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6237 - loss: 1.0943 - val_accuracy: 0.6514 - val_loss: 1.0278
Epoch 8/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6339 - loss: 1.0599 - val_accuracy: 0

,accuracy,loss,val_accuracy,val_loss
19,0.681689,0.928348,0.6884,0.930164
17,0.675400,0.947754,0.6882,0.931728
18,0.676333,0.943682,0.6838,0.942281
16,0.673511,0.952600,0.6834,0.943080
15,0.669378,0.963945,0.6816,0.944531


In [23]:
histDfDropOut35Cifar.sort_values("val_loss", ascending=True).head()

,accuracy,loss,val_accuracy,val_loss
19,0.681689,0.928348,0.6884,0.930164
17,0.675400,0.947754,0.6882,0.931728
18,0.676333,0.943682,0.6838,0.942281
16,0.673511,0.952600,0.6834,0.943080
15,0.669378,0.963945,0.6816,0.944531


In [26]:
modelMaskPlusDropOut35Cifar = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="elu"),
        Masksembles2D(4, 2.0), # adding Masksembles2D
        layers.Dropout(0.1),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Conv2D(64, kernel_size=(3, 3), activation="elu"),
        #Masksembles2D(4, 2.0), # adding Masksembles2D
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Flatten(),
        layers.Dropout(0.35),
        #Masksembles1D(4, 2.), # adding Masksembles1D
        layers.Dense(num_classes, activation="softmax"),
    ]
)

modelMaskPlusDropOut35Cifar.summary()
modelMaskPlusDropOut35Cifar.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
histmodelMaskPlusDropOut35Cifar = modelMaskPlusDropOut35Cifar.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
histmodelMaskPlusDropOut35Cifar = pd.DataFrame(histmodelMaskPlusDropOut35Cifar.history)
histmodelMaskPlusDropOut35Cifar.sort_values("val_accuracy", ascending=False).head()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ masksembles2d_10                │ (None, 30, 30, 32)     │           128 │
│ (Masksembles2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 30, 30, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 10)             │        23,050 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,570 (166.29 KB)

 Trainable params: 42,442 (165.79 KB)

 Non-trainable params: 128 (512.00 B)

Epoch 1/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.2813 - loss: 1.9856 - val_accuracy: 0.4380 - val_loss: 1.5865
Epoch 2/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4653 - loss: 1.5266 - val_accuracy: 0.4964 - val_loss: 1.4359
Epoch 3/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5143 - loss: 1.3917 - val_accuracy: 0.5426 - val_loss: 1.3402
Epoch 4/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5453 - loss: 1.3058 - val_accuracy: 0.5588 - val_loss: 1.2832
Epoch 5/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5641 - loss: 1.2429 - val_accuracy: 0.5324 - val_loss: 1.3274
Epoch 6/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5813 - loss: 1.2145 - val_accuracy: 0.6070 - val_loss: 1.1563
Epoch 7/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5889 - loss: 1.1958 - val_accuracy: 0.5836 - val_loss: 1.2073
Epoch 8/20
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6031 - loss: 1.1468 - val_accuracy: 0

,accuracy,loss,val_accuracy,val_loss
18,0.637511,1.047353,0.6526,1.024173
16,0.635667,1.059088,0.6512,1.028004
17,0.637378,1.055580,0.6416,1.042345
15,0.630622,1.067666,0.6408,1.046408
12,0.622867,1.091132,0.6354,1.070705
